In [ ]:
# Import dependencies
import os
from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [1]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset
from src.model.densnet.tensorflow_dense_net_basic import build_densenet

# Input Data
initial_epoch   = None
resume_training = False
checkpoint_path = None
set_name        = "SimplifiedDenseNetV1"
model_name      = f"{set_name}_2"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

2025-04-20 08:50:14.722422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-20 08:50:14.729304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745131814.737138    4472 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745131814.739584    4472 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745131814.745692    4472 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

2025-04-20 08:50:16.205319: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 1048576
2025-04-20 08:50:53.727414: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
2025-04-20 08:50:57.841188: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 71842 samples [00:41, 1724.17 samples/s]
2025-04-20 08:51:07.182365: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 15431 samples [00:09, 1652.24 samples/s]


In [3]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryCrossentropy(from_logits=False)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

In [4]:
# Model Training
model          = build_densenet(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-04-20 08:51:08 - INFO - 
=== Training Started ===

2025-04-20 08:51:08 - INFO - Batch Size: 128
2025-04-20 08:51:08 - INFO - Optimizer: Adam
2025-04-20 08:51:08 - INFO - 

2025-04-20 08:51:08 - INFO - 
=== Starting Epoch 1 ===



Epoch 1/100


2025-04-20 08:51:29.611586: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:10: Filling up shuffle buffer (this may take a while): 17910 of 25000
2025-04-20 08:51:33.703825: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
I0000 00:00:1745131894.005881    4555 service.cc:152] XLA service 0x78add4005610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745131894.005936    4555 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
2025-04-20 08:51:34.971653: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745131897.665017    4555 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745131962.687392    4555 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

307/561 ━━━━━━━━━━━━━━━━━━━━ 1:04 256ms/step - accuracy: 0.9062 - auc: 0.5332 - f1_score: 0.2264 - loss: 0.8121 - precision: 0.4534 - recall: 0.2177

2025-04-20 08:54:01.082526: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


...

561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - accuracy: 0.9548 - auc: 0.9210 - f1_score: 0.6563 - loss: 0.1581 - precision: 0.8303 - recall: 0.5940 

2025-04-20 12:30:27.847991: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134225408 bytes after encountering the first element of size 134225408 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2025-04-20 12:30:27.848088: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 134225408 bytes after encountering the first element of size 134225408 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2025-04-20 12:30:41 - INFO - 
=== Epoch 69 Summary ===
2025-04-20 12:30:41 - INFO - Time: 186.37s
2025-04-20 12:30:41 - INFO - Training   - accuracy: 0.9549 - auc: 0.9228 - f1_score: 0.6571 - loss: 0.1579 - precision: 0.8304 - recall: 0.5954 - learning_rate: 0.0000
2025-04-20 12:30:41 - INFO - Validation - accu

561/561 ━━━━━━━━━━━━━━━━━━━━ 186s 332ms/step - accuracy: 0.9548 - auc: 0.9210 - f1_score: 0.6563 - loss: 0.1581 - precision: 0.8303 - recall: 0.5940 - val_accuracy: 0.9207 - val_auc: 0.7254 - val_f1_score: 0.4085 - val_loss: 0.3046 - val_precision: 0.5495 - val_recall: 0.3827 - learning_rate: 1.0000e-06


2025-04-20 12:30:41 - INFO - 
=== Training Completed! ===

2025-04-20 12:30:41 - INFO - Final Metrics: accuracy: 0.9549 - auc: 0.9228 - f1_score: 0.6571 - loss: 0.1579 - precision: 0.8304 - recall: 0.5954 - val_accuracy: 0.9207 - val_auc: 0.7254 - val_f1_score: 0.4085 - val_loss: 0.3046 - val_precision: 0.5495 - val_recall: 0.3827



Epoch 69: early stopping
Restoring model weights from the end of the best epoch: 54.
